In [1]:
from tqdm import tqdm_notebook
from tqdm import tqdm

import json
import pandas as pd

from utils.openai_utils import LLMTripletExtractor
from utils.dynamic_index_utils import Aligner

import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

/home/jovyan/chepurova/Text2KG/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("musique_200_test.json", "r") as f:
    ds = json.load(f)

In [3]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [4]:
import string
def normalize(input_string):
    input_string = input_string.lower()
    translator = str.maketrans('', '', string.punctuation)
    return input_string.translate(translator)

In [5]:
id2sample = {}
for elem in ds['data']:
    id2sample[elem['id']] = elem

In [6]:
model_name = 'gpt-4o'
device = 'cuda:1'

aligner = Aligner(device=device, k=2)
extractor = LLMTripletExtractor(model=model_name)

In [7]:
extractor.calculate_cost()

0.0

In [8]:
id2unique_top_entities = {}
id2extracted_entities = {}

# retrieved_triplets = {}

for sample in tqdm(os.listdir('musique_res')):

    df = pd.read_csv("musique_res/" + sample, index_col=0)

    aligner = Aligner(device=device)
    entities = list(set(list(df.subject) + list(df.object)))
    aligner.add_entities(entities, descriptions=['' for _ in range(len(entities))])

    id_ = sample.split(".")[0]
    extracted_entities = extractor.extract_entities_from_question(id2sample[id_]['question'])

    id2extracted_entities[id_] = extracted_entities

    top_entities = aligner.top_entities_by_llm_output(list(extracted_entities.keys()))
    unique_top_entities = []

    for mapped_entities in top_entities.values():
        unique_top_entities.extend(mapped_entities)

    id2unique_top_entities[id_] = unique_top_entities

    

  0%|          | 0/50 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 50/50 [04:14<00:00,  5.10s/it]


In [10]:
retrieved_triplets = {}

for file in os.listdir('musique_res'):
    df = pd.read_csv('musique_res/{}'.format(file), index_col=0)
    idx = file.split('.')[0]

    unique_top_entities = id2unique_top_entities[idx]
    unique_top_entities = list(set(unique_top_entities))

    triplets = df[(df['subject'].apply(lambda x: x in unique_top_entities)) | (df['object'].apply(lambda x: x in unique_top_entities))]

    one_hop_entities = list(triplets['subject']) + list(triplets['object'])
    one_hop_entities = set(one_hop_entities) - set(unique_top_entities)
    
    one_hop_triplets = df[(df['subject'].apply(lambda x: x in one_hop_entities)) | (df['object'].apply(lambda x: x in one_hop_entities))]

    triplets = pd.concat((triplets, one_hop_triplets))

    triplets = triplets.to_records(index=False)

    retrieved_triplets[idx] = triplets


In [11]:
extractor.calculate_cost()

0.113755

In [12]:
llm_answers = []
f1s = []
ems = []
# retrieved_triplets = {}

for file in tqdm(os.listdir('musique_res')):

    df = pd.read_csv("musique_res/" + file, index_col=0)

    aligner = Aligner(device=device)
    entities = list(set(list(df.subject) + list(df.object)))
    aligner.add_entities(entities, descriptions=['' for _ in range(len(entities))])

    sample_id = file.split(".")[0]

    triplets = retrieved_triplets[sample_id]
    question = id2sample[sample_id]['question']
    answer = id2sample[sample_id]['answer']

    triplets = [f"({triplet[0]}, {triplet[1]}, {triplet[2]})" for triplet in triplets]
    triplets = '\n'.join(triplets)
     
    llm_answer = extractor.answer_question(question=question, triplets=triplets)
    llm_answers.append(llm_answer)

    ems.append(llm_answer == answer)
    f1s.append(compute_f1(llm_answer, answer))

    print(llm_answer, answer)

  2%|▏         | 1/50 [00:02<01:52,  2.30s/it]

Rabbi Menachem Mendel Schneersohn followed 3. Dovber Schneuri


  4%|▍         | 2/50 [00:04<01:46,  2.22s/it]

Golden State Warriors Oklahoma City Thunder


  6%|▌         | 3/50 [00:06<01:43,  2.21s/it]

...

15,900 15,900


 98%|█████████▊| 49/50 [02:04<00:02,  2.27s/it]

Min Zhou and Carl L. Bankston III wrote a book about growing up in the same nationality as the man who produced The Wild Women of Chastity Gulch. Min Zhou


100%|██████████| 50/50 [02:07<00:00,  2.55s/it]

The war with King Pyrrhus, whom the Ioannina National Airport is named after, occurred in the years 280-275 BC. 323–272 BC


In [13]:
cost_one_hop = extractor.calculate_cost() - 0.113755
cost_one_hop

0.6433450000000001

In [14]:
import numpy as np
np.mean(ems), np.mean(f1s)

(0.28, 0.3893787878787879)

In [15]:
retrieved_triplets = {}

for file in os.listdir('musique_res'):
    df = pd.read_csv('musique_res/{}'.format(file), index_col=0)
    idx = file.split('.')[0]

    unique_top_entities = id2unique_top_entities[idx]
    unique_top_entities = list(set(unique_top_entities))

    triplets = df[(df['subject'].apply(lambda x: x in unique_top_entities)) | (df['object'].apply(lambda x: x in unique_top_entities))]
    one_hop_entities = list(triplets['subject']) + list(triplets['object'])
    one_hop_entities = set(one_hop_entities) - set(unique_top_entities)
    one_hop_triplets = df[(df['subject'].apply(lambda x: x in one_hop_entities)) | (df['object'].apply(lambda x: x in one_hop_entities))]

    two_hop_entities = list(one_hop_triplets['subject']) + list(one_hop_triplets['object'])

    two_hop_entities = set(two_hop_entities) - set(one_hop_entities) - set(unique_top_entities)
    two_hop_triplets = df[(df['subject'].apply(lambda x: x in two_hop_entities)) | (df['object'].apply(lambda x: x in two_hop_entities))]

    triplets = pd.concat((triplets, one_hop_triplets, two_hop_triplets))

    triplets = triplets.to_records(index=False)

    retrieved_triplets[idx] = triplets


In [16]:
id2unique_top_entities = {}
id2extracted_entities = {}
llm_answers = []
f1s = []
ems = []
# retrieved_triplets = {}

for file in tqdm(os.listdir('musique_res')):

    df = pd.read_csv("musique_res/" + file, index_col=0)

    aligner = Aligner(device=device)
    entities = list(set(list(df.subject) + list(df.object)))
    aligner.add_entities(entities, descriptions=['' for _ in range(len(entities))])

    sample_id = file.split(".")[0]

    triplets = retrieved_triplets[sample_id]
    question = id2sample[sample_id]['question']
    answer = id2sample[sample_id]['answer']

    triplets = [f"({triplet[0]}, {triplet[1]}, {triplet[2]})" for triplet in triplets]
    triplets = '\n'.join(triplets)
     
    llm_answer = extractor.answer_question(question=question, triplets=triplets)
    llm_answers.append(llm_answer)

    ems.append(llm_answer == answer)
    f1s.append(compute_f1(llm_answer, answer))

    print(llm_answer, answer)

  2%|▏         | 1/50 [00:02<02:13,  2.72s/it]

Rabbi Menachem Mendel Schneersohn followed the Rebbe of the Chabad Hasidic movement. Dovber Schneuri


  4%|▍         | 2/50 [00:04<01:50,  2.30s/it]

Golden State Warriors Oklahoma City Thunder


  6%|▌         | 3/50 [00:07<01:53,  2.42s/it]

...

15,900 15,900


 98%|█████████▊| 49/50 [02:14<00:02,  2.48s/it]

Min Zhou and Carl L. Bankston III wrote a book about growing up in the same nationality as the man who produced The Wild Women of Chastity Gulch. Min Zhou


100%|██████████| 50/50 [02:17<00:00,  2.75s/it]

The war with King Pyrrhus, whom the Ioannina National Airport is named after, occurred during the years of the Pyrrhic War, which took place from 280 BC to 275 BC. 323–272 BC


In [19]:
np.mean(ems), np.mean(f1s)

(0.26, 0.34443148148148145)

In [18]:
extractor.calculate_cost() - cost_one_hop

1.065275